# Apply FS in ANN
Condition to improvement only if RMSE on normal regions improves

In [1]:
! pip install pandas
! pip install numpy
! pip install sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

# empty list to append metric values
mae_gen = []
mae_nor = []
mae_spi = []
rmse_gen = []
rmse_nor = []
rmse_spi = []

     |████████████████████████████████| 10.5 MB 23.4 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 69.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.8 MB 21.0 MB/s eta 0:00:01
     |████████████████████████████████| 300 kB 62.3 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=ca2eef3fb4f2a87fd5f7a944e1f117331cca61e93a3b9ae350c70187150e770e
  Stored in directory: /home/ec2-user/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


# Import data & treat it

In [2]:
data = pd.read_csv('Data_set_1_smaller_(1).csv', index_col = 0)

# set predictive window according with tuning best results
data = data.loc[data.index > 2018090000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)


# Import libraries:

In [3]:
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import initializers
from keras import optimizers

# parameters for ANN
splits = 7
epochs = 80

# divide into features and label
X = data.iloc[:, 0:14]
y = data.loc[:, 'Offers']

Using TensorFlow backend.


# Do first prediciton with first feature in list from Linear Regression FS:

In [4]:
# data set to append here (start with first feature added)
X_ = X.loc[:,'PrevDay']

# do first prediction
X_.fillna(method = 'ffill', inplace = True)
y.fillna(method = 'ffill', inplace = True)

X_ = X_.astype('float64')
X_ = X_.round(20)

X_train, X_test, y_train, y_test = train_test_split(
         X_, y, test_size = 0.15, shuffle = False)

X_train = np.array(X_train)
X_train = X_train.reshape(-1, 1)
X_test = np.array(X_test)
X_test = X_test.reshape(-1, 1)

sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

def regressor_tunning(n_hidden = 5, 
                      n_neurons = 40, 
                      kernel_initializer = "he_normal",
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    model.add(Dense(units = n_neurons, input_dim = 1))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dropout(rate = 0.3))
    for layer in range(n_hidden):
        model.add(Dense(units = n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.3))
    model.add(Dense(units = 1, activation = 'linear'))
    optimizer = optimizers.Adamax(lr = 0.001)
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

tscv = TimeSeriesSplit(n_splits = splits)    
regressor = regressor_tunning()

# train model
for train_index, test_index in tscv.split(X_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    regressor.fit(X_train_split, y_train_split,  
                         shuffle = False, 
                         validation_split = 0.2,
                         batch_size = 20, 
                         epochs = epochs)

# make predictions and evaluate for all regions
y_pred = regressor.predict(X_test)

# =============================================================================
# METRICS EVALUATION (1) for the whole test set
# =============================================================================
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

# calculate metrics
rmse_error = mse(y_test, y_pred, squared = False)
mae_error = mae(y_test, y_pred)

# append to list
rmse_gen.append(rmse_error)
mae_gen.append(mae_error)

# =============================================================================
# METRICS EVALUATION (2) on spike regions
# =============================================================================

# download spike indication binary set
y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values

# smal adjustment
y_test = pd.Series(y_test)
y_test.replace(0, 0.0001,inplace = True)

# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]

# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mae_spike = mae(y_test_spike, y_pred_spike)

# append ot lists
rmse_spi.append(rmse_spike)
mae_spi.append(mae_spike)

# =============================================================================
# METRIC EVALUATION (3) on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)

# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mae_normal = mae(y_test_normal, y_pred_normal)

# append to list
rmse_nor.append(rmse_normal)
mae_nor.append(mae_normal)

/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Train on 500 samples, validate on 125 samples
Epoch 1/80
500/500 [==============================] - 7s 15ms/step - loss: 13342.8296 - mse: 13342.8301 - mae: 109.9652 - val_loss: 34649.4106 - val_mse: 34649.4102 - val_mae: 132.8451
Epoch 2/80
500/500 [==============================] - 0s 569us/step - loss: 13250.4004 - mse: 13250.3994 - mae: 109.5460 - val_loss: 34471.7046 - val_mse: 34471.7031 - val_mae: 132.1760
Epoch 3/80
500/500 [==============================] - 0s 558us/step - loss: 13002.3266 - mse: 13002.3271 - mae: 108.4144 - val_loss: 33987.6898 - val_mse: 33987.6914 - val_mae: 130.3369
Epoch 4/80
500/500 [==============================] - 0s 558us/step - loss: 12358.2243 - mse: 12358.2227 - mae: 105.4548 - val_loss: 32738.7081 - val_mse: 32738.7070 - val_mae: 125.4684
Epoch 5/80
500/500 [==============================] - 0s 570us/step - loss: 10790.4475 - mse: 10790.4482 - mae: 97.6580 - val_loss: 29793.4759 - val_mse: 29793.4766 - val_mae: 113.1634
Epoch 6/80
500/500 [======

# Apply loop for the rest of the list of features with condition of improvement:

In [5]:
# features list; order made according to Linear Regression FS
features_list = ['APXP', 
                 'LOLP',  
                 'In_gen',
                 'Ren_R',
                 'DA_imb_France', 
                 'Rene',
                 'ratio_offers_vol',
                 'DA_price_france',
                 'TSDF',
                 'dino_bin',
                 'DA_margin',
                 'Im_Pr']

best_score = rmse_normal

for i in features_list: 
    
   # X_recovery = X_.copy()
    X_recovery = X_
    
    X_ = pd.concat([X_, X.loc[:,i]], axis = 1)
                    
    X_.fillna(method = 'ffill', inplace = True)
    y.fillna(method = 'ffill', inplace = True)

    X_ = X_.astype('float64')
    X_ = X_.round(20)

    X_train, X_test, y_train, y_test = train_test_split(
         X_, y, test_size = 0.15, shuffle = False)
    
    sc_X = MinMaxScaler()
    X_train = sc_X.fit_transform(X_train)
    X_test = sc_X.transform(X_test)

    # possible debug
    X_train = np.nan_to_num(X_train)
    X_test = np.nan_to_num(X_test)

    def regressor_tunning(n_hidden = 5, 
                          n_neurons = 40, 
                          kernel_initializer = "he_normal",
                          bias_initializer = initializers.Ones()):
        model = Sequential()
        model.add(Dense(units = n_neurons, input_dim = len(X_.columns)))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.3))
        for layer in range(n_hidden):
            model.add(Dense(units = n_neurons))
            model.add(keras.layers.LeakyReLU(alpha = 0.2))
            model.add(Dropout(rate = 0.3))
        model.add(Dense(units = 1, activation = 'linear'))
        optimizer = optimizers.Adamax(lr = 0.001)
        model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
        return model

    tscv = TimeSeriesSplit(n_splits = splits)    
    regressor = regressor_tunning()

    # train model
    for train_index, test_index in tscv.split(X_train):
        X_train_split, X_test_split = X_train[train_index], X_train[test_index]
        y_train_split, y_test_split = y_train[train_index], y_train[test_index]
        regressor.fit(X_train_split, y_train_split,  
                             shuffle = False, 
                             validation_split = 0.2,
                             batch_size = 20, 
                             epochs = epochs)

    # make predictions and evaluate for all regions
    y_pred = regressor.predict(X_test)

    # =============================================================================
    # METRICS EVALUATION (1) for the whole test set
    # =============================================================================
    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    # calculate metrics
    rmse_error = mse(y_test, y_pred, squared = False)
    mae_error = mae(y_test, y_pred)

    # append to list
    rmse_gen.append(rmse_error)
    mae_gen.append(mae_error)

    # =============================================================================
    # METRICS EVALUATION (2) on spike regions
    # =============================================================================

    # download spike indication binary set
    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values

    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)

    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]

    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mae_spike = mae(y_test_spike, y_pred_spike)

    # append ot lists
    rmse_spi.append(rmse_spike)
    mae_spi.append(mae_spike)

    # =============================================================================
    # METRIC EVALUATION (3) on normal regions
    # =============================================================================

    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)

    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mae_normal = mae(y_test_normal, y_pred_normal)

    # append to list
    rmse_nor.append(rmse_normal)
    mae_nor.append(mae_normal)

    # condition of improvement for FS
    if best_score < rmse_nor[-1]:
        X_ = X_recovery
    else:
        X_ = X_
        best_score = rmse_normal


Train on 500 samples, validate on 125 samples
Epoch 1/80
500/500 [==============================] - 4s 7ms/step - loss: 13328.6912 - mse: 13328.6914 - mae: 109.9012 - val_loss: 34613.4478 - val_mse: 34613.4492 - val_mae: 132.7116
Epoch 2/80
500/500 [==============================] - 0s 615us/step - loss: 13172.5818 - mse: 13172.5811 - mae: 109.1917 - val_loss: 34309.9562 - val_mse: 34309.9570 - val_mae: 131.5699
Epoch 3/80
500/500 [==============================] - 0s 585us/step - loss: 12727.3288 - mse: 12727.3301 - mae: 107.1292 - val_loss: 33350.7992 - val_mse: 33350.7969 - val_mae: 127.8952
Epoch 4/80
500/500 [==============================] - 0s 595us/step - loss: 11334.0678 - mse: 11334.0674 - mae: 100.5258 - val_loss: 30683.7009 - val_mse: 30683.6992 - val_mae: 117.0720
Epoch 5/80
500/500 [==============================] - 0s 603us/step - loss: 8188.1429 - mse: 8188.1431 - mae: 82.7140 - val_loss: 24728.8058 - val_mse: 24728.8066 - val_mae: 88.2368
Epoch 6/80
500/500 [==========

In [6]:
features_list_index =   ['PrevDay',
                         'APXP', 
                         'LOLP',  
                         'In_gen',
                         'Ren_R',
                         'DA_imb_France', 
                         'Rene',
                         'ratio_offers_vol',
                         'DA_price_france',
                         'TSDF',
                         'dino_bin',
                         'DA_margin',
                         'Im_Pr']

results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,}, index = features_list_index)

results

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
PrevDay,30.726699,21.294778,72.114891,61.039473,17.782678,15.431658
APXP,30.429759,21.596119,69.591476,57.999739,18.658391,16.225872
LOLP,30.605767,20.877213,72.383195,61.226702,17.377950,14.924873
In_gen,30.602583,21.084103,71.982150,60.891433,17.615632,15.211743
Ren_R,30.707674,20.765516,72.611698,61.299822,17.443496,14.785912
DA_imb_France,30.591610,21.202803,71.791700,60.706185,17.708170,15.375280
Rene,30.616220,20.965014,72.360681,61.261438,17.412876,15.020503
ratio_offers_vol,31.283237,21.222581,73.264279,61.346841,18.198226,15.303467
DA_price_france,30.572388,20.439204,73.139244,62.081320,16.835066,14.296177
TSDF,30.379316,21.180851,69.203709,57.352566,18.776453,15.844815


In [7]:
X_.columns

Index(['PrevDay', 'LOLP', 'DA_price_france', 'Im_Pr'], dtype='object')

In [8]:
best_score

16.555087250883144